In [ ]:
# import numpy as np
# from collections import Counter

# def generate_kmers(sequence, k):
#     """Extract all k-mers from a sequence."""
#     return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# def build_kmer_dict(sequences, k):
#     """Create a dictionary mapping each unique k-mer to an index."""
#     kmer_set = set()
#     for seq in sequences:
#         kmer_set.update(generate_kmers(seq, k))
#     return {kmer: idx for idx, kmer in enumerate(sorted(kmer_set))}

# def compute_kmer_feature_matrix(sequences, k, kmer_dict):
#     """Convert sequences into k-mer frequency vectors."""
#     num_samples = len(sequences)
#     num_kmers = len(kmer_dict)
#     feature_matrix = np.zeros((num_samples, num_kmers))

#     for i, seq in enumerate(sequences):
#         kmer_counts = Counter(generate_kmers(seq, k))
#         for kmer, count in kmer_counts.items():
#             if kmer in kmer_dict:
#                 feature_matrix[i, kmer_dict[kmer]] = count
#     return feature_matrix

# def spectrum_kernel_matrix(sequences, k):
#     """Compute the Spectrum Kernel matrix."""
#     kmer_dict = build_kmer_dict(sequences, k)
#     feature_matrix = compute_kmer_feature_matrix(sequences, k, kmer_dict)
#     return feature_matrix @ feature_matrix.T  # Compute the dot product

# # Example usage
# sequences = ["AGCT", "GCTA", "CTAG"]
# k = 2
# K = spectrum_kernel_matrix(sequences, k)
# print(K)


[[3. 2. 2.]
 [2. 3. 2.]
 [2. 2. 3.]]


In [ ]:
# import numpy as np
# import cvxopt

# def train_svm(K, y, C=1.0):
#     """Train an SVM using the precomputed kernel matrix K."""
#     n = len(y)
#     y = y.astype(float).reshape(-1, 1)  # Ensure y is a column vector

#     # Construct the quadratic programming matrices
#     P = cvxopt.matrix(np.outer(y, y) * K)  # P_ij = y_i * y_j * K_ij
#     q = cvxopt.matrix(-np.ones(n))        # q_i = -1
#     G = cvxopt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # Constraints 0 <= α <= C
#     h = cvxopt.matrix(np.hstack((np.zeros(n), C * np.ones(n))))
#     A = cvxopt.matrix(y.T)  # Equality constraint sum(α_i * y_i) = 0
#     b = cvxopt.matrix(0.0)

#     # Solve the quadratic program
#     solution = cvxopt.solvers.qp(P, q, G, h, A, b)
#     alphas = np.ravel(solution['x'])

#     # Support vectors: α > 0
#     sv_indices = alphas > 1e-5
#     support_vectors = np.where(sv_indices)[0]
#     alphas = alphas[sv_indices]
#     support_y = y[sv_indices].flatten()

#     # Compute the bias term (intercept)
#     bias = np.mean(support_y - np.sum(alphas * support_y * K[support_vectors][:, support_vectors], axis=1))

#     return alphas, support_vectors, bias

# def predict_svm(K_test, alphas, support_vectors, support_y, bias):
#     """Make predictions using the trained SVM."""
#     return np.sign(np.sum(alphas * support_y * K_test[:, support_vectors], axis=1) + bias)

# # Example usage
# sequences_train = ["AGCT", "GCTA", "CTAG", "TAGC"]
# sequences_test = ["AGCT", "CTAG"]

# y_train = np.array([1, -1, 1, -1])  # Example labels

# k = 2
# K_train = spectrum_kernel_matrix(sequences_train, k)
# alphas, support_vectors, bias = train_svm(K_train, y_train)

# K_test = spectrum_kernel_matrix(sequences_test + sequences_train, k)[:len(sequences_test), :]
# predictions = predict_svm(K_test, alphas, support_vectors, y_train[support_vectors], bias)

# print("Predictions:", predictions)


     pcost       dcost       gap    pres   dres
 0: -1.7778e+00 -6.2222e+00  4e+00  0e+00  6e-16
 1: -1.8769e+00 -2.1631e+00  3e-01  2e-16  3e-16
 2: -1.9815e+00 -2.0225e+00  4e-02  8e-17  2e-16
 3: -1.9973e+00 -2.0030e+00  6e-03  1e-16  1e-16
 4: -1.9996e+00 -2.0004e+00  8e-04  2e-16  3e-16
 5: -1.9999e+00 -2.0001e+00  1e-04  1e-16  9e-17
 6: -2.0000e+00 -2.0000e+00  2e-05  2e-16  2e-16
 7: -2.0000e+00 -2.0000e+00  2e-06  1e-16  1e-16
 8: -2.0000e+00 -2.0000e+00  3e-07  2e-16  4e-16
Optimal solution found.
Predictions: [ 1. -1.]


Next Steps

    Tune the C parameter.
    Try different values of k (k-mer size).
    Optimize performance (e.g., using sparse representations for large sequences).

In [1]:
import numpy as np
import pandas as pd
import cvxopt
from collections import Counter

# def load_data(seq_file: str, label_file: str) -> tuple:
#     """
#     加载多组CSV文件并合并数据
    
#     参数:
#         seq_files: 多个x.csv文件路径列表（如["x.csv", "x1.csv"]）
#         label_files: 多个y.csv文件路径列表（如["y.csv", "y1.csv"]）
    
#     返回:
#         (sequences, labels): 合并后的序列列表和标签数组
#     """
#     # 合并所有数据
#     df_seq = pd.read_csv(seq_file)
#     df_label = pd.read_csv(label_file)
    
#     merged = pd.merge(df_seq, df_label, on='Id', how='inner')
#     if merged.empty:
#         print(f"警告: {seq_file}和{label_file}中没有匹配的ID")

#     # 提取数据
#     sequences = merged['seq'].values
#     labels = np.where(merged['Bound'] == 1, 1, -1)
#     return sequences, labels


def generate_kmers(sequence, k):
    """Extract all k-mers from a sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

def build_kmer_dict(sequences, k):
    """Create a dictionary mapping each unique k-mer to an index."""
    kmer_set = set()
    for seq in sequences:
        kmer_set.update(generate_kmers(seq, k))
    return {kmer: idx for idx, kmer in enumerate(sorted(kmer_set))}

def compute_kmer_feature_matrix(sequences, k, kmer_dict):
    """Convert sequences into k-mer frequency vectors."""
    num_samples = len(sequences)
    num_kmers = len(kmer_dict)
    feature_matrix = np.zeros((num_samples, num_kmers))

    for i, seq in enumerate(sequences):
        kmer_counts = Counter(generate_kmers(seq, k))
        for kmer, count in kmer_counts.items():
            if kmer in kmer_dict:
                feature_matrix[i, kmer_dict[kmer]] = count
    return feature_matrix

def spectrum_kernel_matrix(sequences1, sequences2, k):
    """Compute the Spectrum Kernel matrix between two sets of sequences."""
    all_sequences = np.concatenate([sequences1, sequences2])
    kmer_dict = build_kmer_dict(all_sequences, k)

    # Convert sequences to feature matrices
    X1 = compute_kmer_feature_matrix(sequences1, k, kmer_dict)
    X2 = compute_kmer_feature_matrix(sequences2, k, kmer_dict)

    return X1 @ X2.T  # Compute the dot product

def train_svm(K, y, C=1.0):
    """Train an SVM using the precomputed kernel matrix K."""
    n = len(y)
    y = y.astype(float).reshape(-1, 1)  # Ensure y is a column vector

    # Construct the quadratic programming matrices
    P = cvxopt.matrix(np.outer(y, y) * K)  # P_ij = y_i * y_j * K_ij
    q = cvxopt.matrix(-np.ones(n))        # q_i = -1
    G = cvxopt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # Constraints 0 <= α <= C
    h = cvxopt.matrix(np.hstack((np.zeros(n), C * np.ones(n))))
    A = cvxopt.matrix(y.T)  # Equality constraint sum(α_i * y_i) = 0
    b = cvxopt.matrix(0.0)

    # Solve the quadratic program
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(solution['x'])

    # Support vectors: α > 0
    sv_indices = alphas > 1e-5
    support_vectors = np.where(sv_indices)[0]
    alphas = alphas[sv_indices]
    support_y = y[sv_indices].flatten()

    # Compute the bias term (intercept)
    bias = np.mean(support_y - np.sum(alphas * support_y * K[support_vectors][:, support_vectors], axis=1))

    return alphas, support_vectors, bias

def predict_svm(K_test, alphas, support_vectors, support_y, bias):
    """Make predictions using the trained SVM."""
    return np.sign(np.sum(alphas * support_y * K_test[:, support_vectors], axis=1) + bias)




# # === Main function to train and predict ===
# def train_and_predict_spectrum_svm(X_train_path, Y_train_path, X_test_path, k=3, C=1.0):
#     """Train an SVM using the Spectrum Kernel and make predictions on test data."""
#     # Load data
#     X_train, Y_train = load_data(X_train_path, Y_train_path)

#     df_pred = pd.read_csv(X_test_path)
#     X_test = df_pred["seq"].values
#     # X_test = load_data(X_test_path)

#     # Compute Kernel matrices
#     K_train = spectrum_kernel_matrix(X_train, X_train, k)
#     K_test = spectrum_kernel_matrix(X_test, X_train, k) # TO CHECK

#     # Train the SVM
#     alphas, support_vectors, bias = train_svm(K_train, Y_train, C)

#     # Predict on test set
#     predictions = predict_svm(K_test, alphas, support_vectors, Y_train[support_vectors], bias)

#     # Convert {-1, 1} predictions to {0, 1}
#     predictions = (predictions + 1) // 2

#     # # Save predictions
#     df_pred["Bound"] = predictions

#     return df_pred
#     # df_pred = pd.DataFrame(predictions, columns=["Bound"])
#     # df_pred.index = np.arange(10000, 10000 + len(predictions))  # Set index starting from 10000
#     # df_pred.to_csv("predictions.csv", index=False)

#     # print("Predictions saved to predictions.csv!")

# # === Run the function with uploaded files ===
# prediction_0 = train_and_predict_spectrum_svm("./data/Xtr0.csv", "./data/Ytr0.csv", "./data/Xte0.csv", k=3, C=1.0)
# prediction_1 = train_and_predict_spectrum_svm("./data/Xtr1.csv", "./data/Ytr1.csv", "./data/Xte1.csv", k = 3, C = 1.0)
# prediction_2 = train_and_predict_spectrum_svm("./data/Xtr2.csv", "./data/Ytr2.csv", ".data/Xte2.csv", k = 3, C=1.0)

In [15]:
X = pd.read_csv("./data/Xtr0.csv")

In [16]:
X

,Id,seq
0,0,GGAGAATCATTTGAACCCGGGAGGTGGAGGTTGCCGTGAGCTGAGA...
1,1,ACCCTGCCTACACCGCGGCGGGGACAGGTGGAGGTTTCAACCCCTG...
2,2,TGCAAATCTGTAAGCATTTCTCAGGCAATGAATTATGTCAACACAA...
3,3,GCGGGACGTGGGCGTCGAGGGTAAGGATATCTGCAGAAGTACTGTC...
4,4,GGAGAATAGCATGTATCCGAGAGGTGGAGCTGGCAGTGAGCCGAGA...
...,...,...
1995,1995,GGAGAATCACTTGAACCCGGGAGGCAGAGGTTGCAGTGAGCCGAGA...
1996,1996,GGAAAAAGACATTCAGGACCACGGACAGATCCAGGACTGCCAGTTG...
1997,1997,TGGCCGGCCCGGCGCCGTCACCGCCCTCAAAAGACATGGCGGCGCC...
1998,1998,GCTTCATCATCACAGTGCAGCAGCTGACTACTTGCTAACTAACGTG...


In [9]:
import numpy as np
import pandas as pd
import cvxopt
import optuna
from collections import Counter
cvxopt.solvers.options["show_progress"] = False


def load_data(seq_file: str, label_file: str) -> tuple:
    """
    加载多组CSV文件并合并数据
    
    参数:
        seq_files: 多个x.csv文件路径列表（如["x.csv", "x1.csv"]）
        label_files: 多个y.csv文件路径列表（如["y.csv", "y1.csv"]）
    
    返回:
        (sequences, labels): 合并后的序列列表和标签数组
    """
    # 合并所有数据
    df_seq = pd.read_csv(seq_file)
    df_label = pd.read_csv(label_file)
    
    merged = pd.merge(df_seq, df_label, on='Id', how='inner')
    if merged.empty:
        print(f"警告: {seq_file}和{label_file}中没有匹配的ID")

    # 提取数据
    sequences = merged['seq'].values
    labels = np.where(merged['Bound'] == 1, 1, -1)
    return sequences, labels

def manual_kfold_split(X, y, n_splits=3, seed=42):
    """Manually splits X and y into K folds for cross-validation."""
    np.random.seed(seed)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    folds = np.array_split(indices, n_splits)
    return folds

def cross_val_score_manual(X, y, k, C, n_splits=3):
    """Performs cross-validation without using sklearn."""
    folds = manual_kfold_split(X, y, n_splits)
    accuracies = []

    for i in range(n_splits):
        # print("i = ", i)
        val_indices = folds[i]  # Current fold is validation set
        train_indices = np.hstack([folds[j] for j in range(n_splits) if j != i])  # Rest are training

        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        # Compute kernel matrices
        K_train = spectrum_kernel_matrix(X_train, X_train, k)
        K_val = spectrum_kernel_matrix(X_val, X_train, k)

        # Train and predict
        alphas, support_vectors, bias = train_svm(K_train, y_train, C)
        predictions = predict_svm(K_val, alphas, support_vectors, y_train[support_vectors], bias)

        # Compute accuracy
        accuracy = np.mean(predictions == y_val)
        accuracies.append(accuracy)

        # print("accuracies: ", accuracies)
    return np.mean(accuracies)

def objective(trial):
    """Objective function for Optuna to optimize k and C."""
    k = trial.suggest_int("k", 2, 5)  # k-mer length between 2 and 6
    C = trial.suggest_loguniform("C", 1e-3, 1e0)  # C in [0.01, 100]

    cross_val = cross_val_score_manual(X_train, Y_train, k, C)

    print(f"C = {C}, k = {k}: {cross_val}")
    return cross_val

def train_and_predict_spectrum_svm(X_train_path, Y_train_path, X_test_path, n_trials=20):
    """Hyperparameter optimization with Optuna, then train and predict."""
    global X_train, Y_train  # Needed for Optuna's objective function
    X_train, Y_train = load_data(X_train_path, Y_train_path)

    df_pred = pd.read_csv(X_test_path)
    X_test = df_pred["seq"].values
    # X_test = load_data(X_test_path)

    # Run Optuna optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # Best hyperparameters
    best_k = study.best_params["k"]
    best_C = study.best_params["C"]
    print(f"Best k: {best_k}, Best C: {best_C}")

    # Train final model with best hyperparameters
    K_train = spectrum_kernel_matrix(X_train, X_train, best_k)
    K_test = spectrum_kernel_matrix(X_test, X_train, best_k)
    alphas, support_vectors, bias = train_svm(K_train, Y_train, best_C)

    # Predict on test set
    predictions = predict_svm(K_test, alphas, support_vectors, Y_train[support_vectors], bias)

    # Convert {-1,1} predictions to {0,1}
    predictions = (predictions + 1) // 2

    df_pred["Bound"] = predictions

    return df_pred

# === Run the function with hyperparameter tuning ===
prediction_0 = train_and_predict_spectrum_svm("./data/Xtr0.csv", "./data/Ytr0.csv", "./data/Xte0.csv", n_trials=20)
prediction_1 = train_and_predict_spectrum_svm("./data/Xtr1.csv", "./data/Ytr1.csv", "./data/Xte1.csv", n_trials=20)
prediction_2 = train_and_predict_spectrum_svm("./data/Xtr2.csv", "./data/Ytr2.csv", "./data/Xte2.csv", n_trials=20)

[I 2025-02-21 19:14:18,324] A new study created in memory with name: no-name-931261fe-99e4-4d94-abe0-95374905a67f
C:\Users\liuce\AppData\Local\Temp\ipykernel_13288\1816469161.py:73: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e0)  # C in [0.01, 100]
[I 2025-02-21 19:14:26,248] Trial 0 finished with value: 0.5699897798848323 and parameters: {'k': 2, 'C': 0.4860433805245759}. Best is trial 0 with value: 0.5699897798848323.


C = 0.4860433805245759, k = 2: 0.5699897798848323


[I 2025-02-21 19:14:33,174] Trial 1 finished with value: 0.5820062941502222 and parameters: {'k': 3, 'C': 0.7765205675026385}. Best is trial 1 with value: 0.5820062941502222.


C = 0.7765205675026385, k = 3: 0.5820062941502222


[I 2025-02-21 19:14:39,748] Trial 2 finished with value: 0.6000138069103587 and parameters: {'k': 4, 'C': 0.037963787831821846}. Best is trial 2 with value: 0.6000138069103587.


...

C = 0.031107143284761617, k = 4: 0.6125023074048562


[I 2025-02-21 19:21:57,746] Trial 18 finished with value: 0.6295118206662434 and parameters: {'k': 5, 'C': 0.007925224469777689}. Best is trial 18 with value: 0.6295118206662434.


C = 0.007925224469777689, k = 5: 0.6295118206662434


[I 2025-02-21 19:22:05,461] Trial 19 finished with value: 0.6255108181644914 and parameters: {'k': 5, 'C': 0.00940072197718209}. Best is trial 18 with value: 0.6295118206662434.


C = 0.00940072197718209, k = 5: 0.6255108181644914
Best k: 5, Best C: 0.007925224469777689


In [19]:
def save_to_csv(df_0, df_1, df_2):
    res = pd.concat([df_0, df_1, df_2], ignore_index = True)
    res["Bound"] = res["Bound"].astype("int")
    res[["Id", "Bound"]].to_csv("./results/predictions.csv", index = False)
    return
    # df_pred.to_csv("predictions.csv"

In [20]:
save_to_csv(prediction_0, prediction_1, prediction_2)